In [2]:
import os
import time
import torch
import argparse
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim
import torch.utils.data
from RGBBranch import RGBBranch
from SemBranch import SemBranch
from SASceneNet import SASceneNet
from Libs.Datasets.SUN397Dataset import SUN397Dataset
from Libs.Utils import utils
from Libs.Utils.torchsummary import torchsummary
import numpy as np
import yaml
from tabulate import tabulate
import csv
from PIL import Image
import matplotlib.pyplot as plt

# Save the results for each sample

In [3]:
def evaluationDataLoader(dataloader, model, set):
    batch_time = utils.AverageMeter()
    losses = utils.AverageMeter()
    top1 = utils.AverageMeter()
    top2 = utils.AverageMeter()
    top5 = utils.AverageMeter()

    ClassTPs_Top1 = torch.zeros(1, len(classes), dtype=torch.uint8).cuda()
    ClassTPs_Top2 = torch.zeros(1, len(classes), dtype=torch.uint8).cuda()
    ClassTPs_Top5 = torch.zeros(1, len(classes), dtype=torch.uint8).cuda()
    Predictions = np.zeros(len(dataloader))
#     print('dataloader length: {}'.format(len(dataloader)))
    Predictions_conf = np.zeros([len(dataloader),397])
    SceneGTLabels = np.zeros(len(dataloader))

    # Extract batch size
    batch_size = CONFIG['VALIDATION']['BATCH_SIZE']['TEST']

    # Start data time
    data_time_start = time.time()

    with torch.no_grad():
        for i, (mini_batch) in enumerate(dataloader):
            start_time = time.time()
            if USE_CUDA:
                RGB_image = mini_batch['Image'].cuda()
                semantic_mask = mini_batch['Semantic'].cuda()
                semantic_scores = mini_batch['Semantic Scores'].cuda()
                sceneLabelGT = mini_batch['Scene Index'].cuda()

            if set is 'Validation' and CONFIG['VALIDATION']['TEN_CROPS']:
                # Fuse batch size and ncrops to set the input for the network
                bs, ncrops, c_img, h, w = RGB_image.size()
                RGB_image = RGB_image.view(-1, c_img, h, w)

                bs, ncrops, c_sem, h, w = semantic_mask.size()
                semantic_mask = semantic_mask.view(-1, c_sem, h, w)

                bs, ncrops, c_sem, h, w = semantic_scores.size()
                semantic_scores = semantic_scores.view(-1, c_sem, h, w)

            # Create tensor of probabilities from semantic_mask
            semanticTensor = utils.make_one_hot(semantic_mask, semantic_scores, C=CONFIG['DATASET']['N_CLASSES_SEM'])

            # Model Forward
            outputSceneLabel, feature_conv, outputSceneLabelRGB, outputSceneLabelSEM = model(RGB_image, semanticTensor)

            if set is 'Validation' and CONFIG['VALIDATION']['TEN_CROPS']:
                # Average results over the 10 crops
                outputSceneLabel = outputSceneLabel.view(bs, ncrops, -1).mean(1)
                outputSceneLabelRGB = outputSceneLabelRGB.view(bs, ncrops, -1).mean(1)
                outputSceneLabelSEM = outputSceneLabelSEM.view(bs, ncrops, -1).mean(1)

            if batch_size is 1:
                if set is 'Validation' and CONFIG['VALIDATION']['TEN_CROPS']:
                    feature_conv = torch.unsqueeze(feature_conv[4, :, :, :], 0)
                    RGB_image = torch.unsqueeze(RGB_image[4, :, :, :], 0)

                # Obtain 10 most scored predicted scene index
                Ten_Predictions = utils.obtainPredictedClasses(outputSceneLabel)
#                 print(np.shape(outputSceneLabel.cpu().numpy()[0]))
#                 print(outputSceneLabel.cpu().numpy()[0])

                # Save predicted label and ground-truth label
                Predictions[i] = Ten_Predictions[0]
                Predictions_conf[i,:] = np.reshape(outputSceneLabel.cpu().numpy()[0],[1,397])
                SceneGTLabels[i] = sceneLabelGT.item()

                # Compute activation maps
                # utils.saveActivationMap(model, feature_conv, Ten_Predictions, sceneLabelGT,
                #                         RGB_image, classes, i, set, save=True)

            # Compute class accuracy
            ClassTPs = utils.getclassAccuracy(outputSceneLabel, sceneLabelGT, len(classes), topk=(1, 2, 5))
            ClassTPs_Top1 += ClassTPs[0]
            ClassTPs_Top2 += ClassTPs[1]
            ClassTPs_Top5 += ClassTPs[2]

            # Compute Loss
            loss = model.loss(outputSceneLabel, sceneLabelGT)

            # Measure Top1, Top2 and Top5 accuracy
            prec1, prec2, prec5 = utils.accuracy(outputSceneLabel.data, sceneLabelGT, topk=(1, 2, 5))

            # Update values
            losses.update(loss.item(), batch_size)
            top1.update(prec1.item(), batch_size)
            top2.update(prec2.item(), batch_size)
            top5.update(prec5.item(), batch_size)

            # Measure batch elapsed time
            batch_time.update(time.time() - start_time)

            # Print information
            if i % CONFIG['VALIDATION']['PRINT_FREQ'] == 0:
                print('Testing {} set batch: [{}/{}]\t'
                      'Batch Time {batch_time.val:.3f} (avg: {batch_time.avg:.3f})\t'
                      'Loss {loss.val:.3f} (avg: {loss.avg:.3f})\t'
                      'Prec@1 {top1.val:.3f} (avg: {top1.avg:.3f})\t'
                      'Prec@2 {top2.val:.3f} (avg: {top2.avg:.3f})\t'
                      'Prec@5 {top5.val:.3f} (avg: {top5.avg:.3f})'.
                      format(set, i, len(dataloader), set, batch_time=batch_time, loss=losses,
                             top1=top1, top2=top2, top5=top5))
                

        ClassTPDic = {'Top1': ClassTPs_Top1.cpu().numpy(),
                      'Top2': ClassTPs_Top2.cpu().numpy(), 'Top5': ClassTPs_Top5.cpu().numpy()}

        print('Elapsed time for {} set evaluation {time:.3f} seconds'.format(set, time=time.time() - data_time_start))
        print("")
#         print(np.shape(Predictions))
        if batch_size is 1:
            # Save predictions and Scene GT in txt file
            np.savetxt(CONFIG['EXP']['OUTPUT_DIR'] + '/' + set + '_Predictions.csv', Predictions, delimiter="," )
            np.savetxt(CONFIG['EXP']['OUTPUT_DIR'] + '/' + set + '_Predictions_conf.csv',Predictions_conf, delimiter=",")
            np.savetxt(CONFIG['EXP']['OUTPUT_DIR'] + '/' + set + '_GT.csv', SceneGTLabels, delimiter=",")

        return top1.avg, top2.avg, top5.avg, losses.avg, ClassTPDic


In [4]:
config_path = './Config/config_SUN397.yaml'
CONFIG = yaml.safe_load(open(config_path, 'r'))
USE_CUDA = torch.cuda.is_available()

# Instantiate network
if CONFIG['MODEL']['ONLY_RGB']:
    print('Evaluating ONLY RGB branch')
    print('Selected RGB backbone architecture: ' + CONFIG['MODEL']['ARCH'])
    model = RGBBranch(arch=CONFIG['MODEL']['ARCH'], scene_classes=CONFIG['DATASET']['N_CLASSES_SCENE'])
elif CONFIG['MODEL']['ONLY_SEM']:
    print('Evaluating ONLY SEM branch')
    model = SemBranch(scene_classes=CONFIG['DATASET']['N_CLASSES_SCENE'], semantic_classes=CONFIG['DATASET']['N_CLASSES_SEM'])
else:
    print('Evaluating complete model')
    print('Selected RG backbone architecture: ' + CONFIG['MODEL']['ARCH'])
    model = SASceneNet(arch=CONFIG['MODEL']['ARCH'], scene_classes=CONFIG['DATASET']['N_CLASSES_SCENE'], semantic_classes=CONFIG['DATASET']['N_CLASSES_SEM'])

Evaluating complete model
Selected RG backbone architecture: ResNet-50


In [5]:
# Load the trained model
completePath = CONFIG['MODEL']['PATH'] + CONFIG['MODEL']['NAME'] + '.pth.tar'
if os.path.isfile(completePath):
    print("Loading model {} from path {}...".format(CONFIG['MODEL']['NAME'], completePath))
    checkpoint = torch.load(completePath)
    best_prec1 = checkpoint['best_prec1']
    model.load_state_dict(checkpoint['state_dict'])
    print("Loaded model {} from path {}.".format(CONFIG['MODEL']['NAME'], completePath))
    print("     Epochs {}".format(checkpoint['epoch']))
    print("     Single crop reported precision {}".format(best_prec1))
else:
    print("No checkpoint found at '{}'. Check configuration file MODEL field".format(completePath))
    quit()

Loading model SAScene_ResNet50_SUN from path ./Data/Model Zoo/SUN397/SAScene_ResNet50_SUN.pth.tar...
Loaded model SAScene_ResNet50_SUN from path ./Data/Model Zoo/SUN397/SAScene_ResNet50_SUN.pth.tar.
     Epochs 70
     Single crop reported precision 71.95781220915568


In [6]:
# Move Model to GPU an set it to evaluation mode
if USE_CUDA:
    model.cuda()
cudnn.benchmark = USE_CUDA
model.eval()

print('-' * 65)
print('Loading dataset {}...'.format(CONFIG['DATASET']['NAME']))

traindir = os.path.join(CONFIG['DATASET']['ROOT'], CONFIG['DATASET']['NAME'])
valdir = os.path.join(CONFIG['DATASET']['ROOT'], CONFIG['DATASET']['NAME'])

-----------------------------------------------------------------
Loading dataset SUN397...


In [7]:

train_dataset = SUN397Dataset(traindir, "train")
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=CONFIG['VALIDATION']['BATCH_SIZE']['TRAIN'],
                                           shuffle=False, num_workers=CONFIG['DATALOADER']['NUM_WORKERS'], pin_memory=True)

val_dataset = SUN397Dataset(valdir, "val", tencrops=CONFIG['VALIDATION']['TEN_CROPS'])
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=CONFIG['VALIDATION']['BATCH_SIZE']['TEST'],
                                         shuffle=False, num_workers=CONFIG['DATALOADER']['NUM_WORKERS'], pin_memory=True)

classes = train_dataset.classes
# classes

In [8]:
# Check if OUTPUT_DIR exists and if not create it
if not os.path.exists(CONFIG['EXP']['OUTPUT_DIR']):
    os.makedirs(CONFIG['EXP']['OUTPUT_DIR'])
    
# start the inference
start = time.time()
val_top1, val_top2, val_top5, val_loss, val_ClassTPDic = evaluationDataLoader(val_loader, model, set='Validation')
end = time.time()
elapsed_time = end - start
print('Inferece took {} for {} samples'.format(elapsed_time, len(val_dataset)))

Testing Validation set batch: [0/19850]	Batch Time 2.709 (avg: 2.709)	Loss 0.900 (avg: 0.900)	Prec@1 100.000 (avg: 100.000)	Prec@2 100.000 (avg: 100.000)	Prec@5 100.000 (avg: 100.000)
Testing Validation set batch: [10/19850]	Batch Time 0.062 (avg: 0.303)	Loss 0.860 (avg: 2.438)	Prec@1 100.000 (avg: 45.455)	Prec@2 100.000 (avg: 54.545)	Prec@5 100.000 (avg: 72.727)
Testing Validation set batch: [20/19850]	Batch Time 0.065 (avg: 0.188)	Loss 0.588 (avg: 2.392)	Prec@1 100.000 (avg: 42.857)	Prec@2 100.000 (avg: 47.619)	Prec@5 100.000 (avg: 76.190)
Testing Validation set batch: [30/19850]	Batch Time 0.065 (avg: 0.148)	Loss 1.042 (avg: 1.943)	Prec@1 0.000 (avg: 48.387)	Prec@2 100.000 (avg: 58.065)	Prec@5 100.000 (avg: 83.871)
Testing Validation set batch: [40/19850]	Batch Time 0.061 (avg: 0.127)	Loss 3.874 (avg: 2.141)	Prec@1 0.000 (avg: 43.902)	Prec@2 0.000 (avg: 53.659)	Prec@5 0.000 (avg: 80.488)
Testing Validation set batch: [50/19850]	Batch Time 0.065 (avg: 0.115)	Loss 0.588 (avg: 2.100)	P

Testing Validation set batch: [460/19850]	Batch Time 0.064 (avg: 0.069)	Loss 0.007 (avg: 1.115)	Prec@1 100.000 (avg: 70.499)	Prec@2 100.000 (avg: 81.996)	Prec@5 100.000 (avg: 91.974)
Testing Validation set batch: [470/19850]	Batch Time 0.064 (avg: 0.069)	Loss 0.327 (avg: 1.116)	Prec@1 100.000 (avg: 70.064)	Prec@2 100.000 (avg: 82.166)	Prec@5 100.000 (avg: 92.144)
Testing Validation set batch: [480/19850]	Batch Time 0.065 (avg: 0.069)	Loss 3.700 (avg: 1.116)	Prec@1 0.000 (avg: 69.647)	Prec@2 0.000 (avg: 82.121)	Prec@5 100.000 (avg: 92.308)
Testing Validation set batch: [490/19850]	Batch Time 0.064 (avg: 0.069)	Loss 3.279 (avg: 1.125)	Prec@1 0.000 (avg: 69.246)	Prec@2 0.000 (avg: 81.670)	Prec@5 100.000 (avg: 92.464)
Testing Validation set batch: [500/19850]	Batch Time 0.064 (avg: 0.069)	Loss 0.030 (avg: 1.121)	Prec@1 100.000 (avg: 69.261)	Prec@2 100.000 (avg: 81.836)	Prec@5 100.000 (avg: 92.615)
Testing Validation set batch: [510/19850]	Batch Time 0.067 (avg: 0.069)	Loss 0.007 (avg: 1.10

Testing Validation set batch: [910/19850]	Batch Time 0.064 (avg: 0.067)	Loss 1.135 (avg: 1.078)	Prec@1 0.000 (avg: 69.484)	Prec@2 100.000 (avg: 83.315)	Prec@5 100.000 (avg: 93.194)
Testing Validation set batch: [920/19850]	Batch Time 0.064 (avg: 0.067)	Loss 0.479 (avg: 1.085)	Prec@1 100.000 (avg: 69.273)	Prec@2 100.000 (avg: 83.170)	Prec@5 100.000 (avg: 93.051)
Testing Validation set batch: [930/19850]	Batch Time 0.067 (avg: 0.067)	Loss 0.176 (avg: 1.099)	Prec@1 100.000 (avg: 68.851)	Prec@2 100.000 (avg: 82.707)	Prec@5 100.000 (avg: 92.911)
Testing Validation set batch: [940/19850]	Batch Time 0.067 (avg: 0.067)	Loss 0.424 (avg: 1.112)	Prec@1 100.000 (avg: 68.438)	Prec@2 100.000 (avg: 82.253)	Prec@5 100.000 (avg: 92.880)
Testing Validation set batch: [950/19850]	Batch Time 0.066 (avg: 0.067)	Loss 0.013 (avg: 1.107)	Prec@1 100.000 (avg: 68.665)	Prec@2 100.000 (avg: 82.440)	Prec@5 100.000 (avg: 92.955)
Testing Validation set batch: [960/19850]	Batch Time 0.067 (avg: 0.067)	Loss 1.479 (avg

Testing Validation set batch: [1360/19850]	Batch Time 0.064 (avg: 0.067)	Loss 0.063 (avg: 1.018)	Prec@1 100.000 (avg: 70.242)	Prec@2 100.000 (avg: 84.423)	Prec@5 100.000 (avg: 94.122)
Testing Validation set batch: [1370/19850]	Batch Time 0.066 (avg: 0.067)	Loss 6.562 (avg: 1.022)	Prec@1 0.000 (avg: 70.241)	Prec@2 0.000 (avg: 84.318)	Prec@5 0.000 (avg: 94.092)
Testing Validation set batch: [1380/19850]	Batch Time 0.066 (avg: 0.067)	Loss 0.274 (avg: 1.018)	Prec@1 100.000 (avg: 70.384)	Prec@2 100.000 (avg: 84.359)	Prec@5 100.000 (avg: 94.135)
Testing Validation set batch: [1390/19850]	Batch Time 0.070 (avg: 0.067)	Loss 0.030 (avg: 1.020)	Prec@1 100.000 (avg: 70.309)	Prec@2 100.000 (avg: 84.328)	Prec@5 100.000 (avg: 94.033)
Testing Validation set batch: [1400/19850]	Batch Time 0.065 (avg: 0.067)	Loss 0.833 (avg: 1.021)	Prec@1 100.000 (avg: 70.236)	Prec@2 100.000 (avg: 84.297)	Prec@5 100.000 (avg: 94.076)
Testing Validation set batch: [1410/19850]	Batch Time 0.068 (avg: 0.067)	Loss 0.007 (a

Testing Validation set batch: [1810/19850]	Batch Time 0.066 (avg: 0.067)	Loss 1.037 (avg: 0.949)	Prec@1 0.000 (avg: 72.281)	Prec@2 100.000 (avg: 86.030)	Prec@5 100.000 (avg: 94.809)
Testing Validation set batch: [1820/19850]	Batch Time 0.067 (avg: 0.067)	Loss 0.003 (avg: 0.948)	Prec@1 100.000 (avg: 72.268)	Prec@2 100.000 (avg: 86.052)	Prec@5 100.000 (avg: 94.838)
Testing Validation set batch: [1830/19850]	Batch Time 0.067 (avg: 0.067)	Loss 0.464 (avg: 0.946)	Prec@1 100.000 (avg: 72.365)	Prec@2 100.000 (avg: 86.128)	Prec@5 100.000 (avg: 94.866)
Testing Validation set batch: [1840/19850]	Batch Time 0.066 (avg: 0.067)	Loss 0.058 (avg: 0.943)	Prec@1 100.000 (avg: 72.461)	Prec@2 100.000 (avg: 86.203)	Prec@5 100.000 (avg: 94.894)
Testing Validation set batch: [1850/19850]	Batch Time 0.066 (avg: 0.067)	Loss 0.192 (avg: 0.940)	Prec@1 100.000 (avg: 72.609)	Prec@2 100.000 (avg: 86.278)	Prec@5 100.000 (avg: 94.922)
Testing Validation set batch: [1860/19850]	Batch Time 0.067 (avg: 0.067)	Loss 1.25

...

Testing Validation set batch: [18010/19850]	Batch Time 0.071 (avg: 0.071)	Loss 0.471 (avg: 0.934)	Prec@1 100.000 (avg: 73.572)	Prec@2 100.000 (avg: 86.142)	Prec@5 100.000 (avg: 94.775)
Testing Validation set batch: [18020/19850]	Batch Time 0.073 (avg: 0.071)	Loss 0.826 (avg: 0.934)	Prec@1 100.000 (avg: 73.581)	Prec@2 100.000 (avg: 86.149)	Prec@5 100.000 (avg: 94.778)
Testing Validation set batch: [18030/19850]	Batch Time 0.071 (avg: 0.071)	Loss 0.000 (avg: 0.934)	Prec@1 100.000 (avg: 73.590)	Prec@2 100.000 (avg: 86.152)	Prec@5 100.000 (avg: 94.776)
Testing Validation set batch: [18040/19850]	Batch Time 0.071 (avg: 0.071)	Loss 0.000 (avg: 0.933)	Prec@1 100.000 (avg: 73.599)	Prec@2 100.000 (avg: 86.154)	Prec@5 100.000 (avg: 94.779)
Testing Validation set batch: [18050/19850]	Batch Time 0.073 (avg: 0.071)	Loss 0.082 (avg: 0.933)	Prec@1 100.000 (avg: 73.608)	Prec@2 100.000 (avg: 86.156)	Prec@5 100.000 (avg: 94.776)
Testing Validation set batch: [18060/19850]	Batch Time 0.071 (avg: 0.071)	L

Testing Validation set batch: [18460/19850]	Batch Time 0.070 (avg: 0.071)	Loss 2.695 (avg: 0.925)	Prec@1 0.000 (avg: 73.842)	Prec@2 100.000 (avg: 86.312)	Prec@5 100.000 (avg: 94.843)
Testing Validation set batch: [18470/19850]	Batch Time 0.071 (avg: 0.071)	Loss 0.880 (avg: 0.925)	Prec@1 100.000 (avg: 73.845)	Prec@2 100.000 (avg: 86.314)	Prec@5 100.000 (avg: 94.846)
Testing Validation set batch: [18480/19850]	Batch Time 0.072 (avg: 0.071)	Loss 0.463 (avg: 0.925)	Prec@1 100.000 (avg: 73.854)	Prec@2 100.000 (avg: 86.321)	Prec@5 100.000 (avg: 94.849)
Testing Validation set batch: [18490/19850]	Batch Time 0.072 (avg: 0.071)	Loss 1.157 (avg: 0.925)	Prec@1 0.000 (avg: 73.847)	Prec@2 100.000 (avg: 86.328)	Prec@5 100.000 (avg: 94.852)
Testing Validation set batch: [18500/19850]	Batch Time 0.072 (avg: 0.071)	Loss 2.976 (avg: 0.925)	Prec@1 0.000 (avg: 73.850)	Prec@2 100.000 (avg: 86.336)	Prec@5 100.000 (avg: 94.854)
Testing Validation set batch: [18510/19850]	Batch Time 0.072 (avg: 0.071)	Loss 1.

Testing Validation set batch: [18910/19850]	Batch Time 0.070 (avg: 0.071)	Loss 0.074 (avg: 0.921)	Prec@1 100.000 (avg: 73.973)	Prec@2 100.000 (avg: 86.399)	Prec@5 100.000 (avg: 94.902)
Testing Validation set batch: [18920/19850]	Batch Time 0.074 (avg: 0.071)	Loss 0.722 (avg: 0.920)	Prec@1 100.000 (avg: 73.987)	Prec@2 100.000 (avg: 86.407)	Prec@5 100.000 (avg: 94.905)
Testing Validation set batch: [18930/19850]	Batch Time 0.071 (avg: 0.071)	Loss 0.311 (avg: 0.920)	Prec@1 100.000 (avg: 74.000)	Prec@2 100.000 (avg: 86.414)	Prec@5 100.000 (avg: 94.908)
Testing Validation set batch: [18940/19850]	Batch Time 0.072 (avg: 0.071)	Loss 0.080 (avg: 0.919)	Prec@1 100.000 (avg: 74.014)	Prec@2 100.000 (avg: 86.421)	Prec@5 100.000 (avg: 94.911)
Testing Validation set batch: [18950/19850]	Batch Time 0.069 (avg: 0.071)	Loss 0.012 (avg: 0.919)	Prec@1 100.000 (avg: 74.022)	Prec@2 100.000 (avg: 86.428)	Prec@5 100.000 (avg: 94.913)
Testing Validation set batch: [18960/19850]	Batch Time 0.075 (avg: 0.071)	L

Testing Validation set batch: [19360/19850]	Batch Time 0.071 (avg: 0.071)	Loss 0.000 (avg: 0.916)	Prec@1 100.000 (avg: 74.092)	Prec@2 100.000 (avg: 86.493)	Prec@5 100.000 (avg: 94.938)
Testing Validation set batch: [19370/19850]	Batch Time 0.072 (avg: 0.071)	Loss 0.001 (avg: 0.916)	Prec@1 100.000 (avg: 74.100)	Prec@2 100.000 (avg: 86.495)	Prec@5 100.000 (avg: 94.936)
Testing Validation set batch: [19380/19850]	Batch Time 0.071 (avg: 0.071)	Loss 0.005 (avg: 0.916)	Prec@1 100.000 (avg: 74.114)	Prec@2 100.000 (avg: 86.502)	Prec@5 100.000 (avg: 94.938)
Testing Validation set batch: [19390/19850]	Batch Time 0.071 (avg: 0.071)	Loss 0.003 (avg: 0.915)	Prec@1 100.000 (avg: 74.127)	Prec@2 100.000 (avg: 86.509)	Prec@5 100.000 (avg: 94.941)
Testing Validation set batch: [19400/19850]	Batch Time 0.070 (avg: 0.071)	Loss 0.288 (avg: 0.915)	Prec@1 100.000 (avg: 74.135)	Prec@2 100.000 (avg: 86.511)	Prec@5 100.000 (avg: 94.938)
Testing Validation set batch: [19410/19850]	Batch Time 0.073 (avg: 0.071)	L

Testing Validation set batch: [19810/19850]	Batch Time 0.074 (avg: 0.071)	Loss 0.318 (avg: 0.912)	Prec@1 100.000 (avg: 74.247)	Prec@2 100.000 (avg: 86.578)	Prec@5 100.000 (avg: 94.952)
Testing Validation set batch: [19820/19850]	Batch Time 0.071 (avg: 0.071)	Loss 0.120 (avg: 0.912)	Prec@1 100.000 (avg: 74.255)	Prec@2 100.000 (avg: 86.585)	Prec@5 100.000 (avg: 94.955)
Testing Validation set batch: [19830/19850]	Batch Time 0.071 (avg: 0.071)	Loss 0.493 (avg: 0.912)	Prec@1 100.000 (avg: 74.252)	Prec@2 100.000 (avg: 86.582)	Prec@5 100.000 (avg: 94.957)
Testing Validation set batch: [19840/19850]	Batch Time 0.071 (avg: 0.071)	Loss 3.229 (avg: 0.912)	Prec@1 0.000 (avg: 74.255)	Prec@2 0.000 (avg: 86.583)	Prec@5 100.000 (avg: 94.960)
Elapsed time for Validation set evaluation 1414.352 seconds

Inferece took 1421.1517419815063 for 19850 samples


In [9]:
dts = []
gts = []
with open('./Results_SUN_0/Validation_Predictions.csv', 'r') as file:
    reader = csv.reader(file)
    for row in reader:
        dts.append(row)

with open('./Results_SUN_0/Validation_GT.csv', 'r') as file:
    reader = csv.reader(file)
    for row in reader:
        gts.append(row)

for i in range (len(gts)):
    gts[i][0] = int(float(gts[i][0]))
for i in range (len(dts)):
    dts[i][0] = int(float(dts[i][0]))
print('there are {} predictions'.format(len(dts)))
print('there are {} ground truth'.format(len(gts)))


there are 19850 predictions
there are 19850 ground truth


In [10]:
# classify val data based on categories
gts_classes = {}
for i in range(397):
    gts_classes[i] = []
for i in range(len(gts)):
    gts_classes[gts[i][0]].append(i)

In [11]:
dts_raw = []
import csv
with open('./Results_SUN_0/Validation_Predictions_conf.csv', 'r') as file:
    reader = csv.reader(file)
    for row in reader:
        dts_raw.append(row)
for i in range(len(dts_raw)):
    dt = dts_raw[i]
    dts_raw[i] = [float(a) for a in dt]
print('there are {} predictions_conf'.format(len(dts_raw)))

there are 19850 predictions_conf


# Level4: Evaluation

In [12]:
def top1_accuracy(dts,gts):
    total = 0
    accurate = 0
    precision_total = 0
    pc_total = [0 for i in range(397)]
    pc_accurate = [0 for i in range(397)]
    pc_precision_total = [0 for i in range(397)]
    for i in range (len(gts)):
        if gts[i][0] == dts[i][0]:
            accurate +=1
            pc_accurate[gts[i][0]] +=1
        total +=1
        pc_total[gts[i][0]] +=1
        pc_precision_total[dts[i][0]] +=1
    return accurate/total, pc_total, pc_accurate, pc_precision_total

def top1_accuracy_highLevel(dts,gts):
    total = 0
    accurate = 0
    precision_total = 0
    for i in range(len(gts)):
        pc_total = [0 for i in range(len(gts[0]))]
        pc_accurate = [0 for i in range(len(gts[0]))]
        pc_precision_total = [0 for i in range(len(gts[0]))]
        break
        
    for i in range(len(gts)):
        for j in range(len(gts[i])):
            if dts[i][j] == 1:
                pc_precision_total[j] +=1
            if gts[i][j] == 1:
                pc_total[j]+=1
                total+=1
                if dts[i][j] == 1:
                    pc_accurate[j] +=1
                    accurate += 1

    acc = accurate/total
#     pc_acc = [pc_accurate[i]/pc_total[i] for i in range(len(pc_total))]
    
    return acc, pc_total,pc_accurate,pc_precision_total

def compute_f1 (pre,recall):
    f1 = 2 * (pre * recall) / (pre + recall)
    f1  = round(f1,2)
    return f1

In [13]:
top1,pc_total_4,pc_accurate_4,pc_precision_total_4 = top1_accuracy(dts,gts)
print('Top-1 accuracy at level 4 is {}'.format(float(top1)))
pc_recall_4 = []
pc_precision_4 = []
for i in range (len(pc_total_4)):
    recall_4 = float(pc_accurate_4[i]/pc_total_4[i])
    recall_4  = round(recall_4,2)
    pc_recall_4.append(recall_4)

for i in range (len(pc_precision_total_4)):
    pre_4 = float(pc_accurate_4[i]/pc_precision_total_4[i])
    pre_4  = round(pre_4,2)
    pc_precision_4.append(pre_4)
    

Top-1 accuracy at level 4 is 0.7425188916876574


In [14]:
table_4 = [[classes[i],pc_total_4[i],pc_accurate_4[i],pc_recall_4[i],pc_precision_4[i],compute_f1(pc_recall_4[i],pc_precision_4[i])] for i in range(365)]
# def takeThird(elem):
#     return elem[3]
# table.sort(key = takeThird,reverse = True)
print(tabulate(table_4, headers=['class', 'total number','correct preditions','recall','precision','f1 core']))

class                          total number    correct preditions    recall    precision    f1 core
---------------------------  --------------  --------------------  --------  -----------  ---------
abbey                                    50                    21      0.42         0.58       0.49
airplane_cabin                           50                    46      0.92         0.92       0.92
airport_terminal                         50                    26      0.52         0.58       0.55
alley                                    50                    36      0.72         0.82       0.77
amphitheater                             50                    44      0.88         0.98       0.93
amusement_arcade                         50                    41      0.82         0.72       0.77
amusement_park                           50                    37      0.74         0.95       0.83
anechoic_chamber                         50                    42      0.84         0.88       0.86


# level 1: Evaluation

In [15]:
lv1_names= ['indoor','outdoor,natural','outdoor, man-made']
level1 = {}
with open('./SUN397_level1.csv', 'r') as file:
    reader = csv.reader(file)
    count = 0
    for row in reader:
        level1[count]=[int(row[a]) for a in range(len(row))]
        count +=1
# load the class label
classes = tuple(classes)

# load ground truth
gts_lv1 = []
for i in range(len(gts)):
    value = gts[i][0]
    gts_lv1.append(level1[value]) 

In [16]:
# mapping dts to level1:
temp = []
for label in level1.keys():
    if level1[label] not in temp:
        temp.append(level1[label])


lv1 = {}
for key in level1.keys():
    lv1[key] = temp.index(level1[key])
# lv1 mapping original class_id to level1_id  

lv1_classes = {}
for i in range(len(temp)):
    lv1_classes[i] = temp[i]

dts_lv1 = []
for i in range(len(dts)):
    temp =lv1_classes[lv1[dts[i][0]]]
    dts_lv1.append(temp)

In [17]:
top1,pc_total,pc_accurate, pc_precision_total = top1_accuracy_highLevel(dts_lv1,gts_lv1)
print('Top-1 accuracy of at level-1 is {}'.format(float(top1)))
for i in range(len(pc_total)):
    total = pc_total[i]
    acc = float(pc_accurate[i]/pc_total[i])
    name = lv1_names[i]
    print('{} samples in: {}. The accurate is {:.2f}'.format(total,name,acc))

Top-1 accuracy of at level-1 is 0.9732723112128147
8850 samples in: indoor. The accurate is 0.99
4450 samples in: outdoor,natural. The accurate is 0.96
8550 samples in: outdoor, man-made. The accurate is 0.97


In [18]:
pc_total_1 = [0 for i in range(397)]
pc_accurate_1 = [0 for i in range(397)]
pc_precision_total_1 = [0 for i in range(397)]
for cat in gts_classes.keys():
    dts_temp = {}
    gts_temp = {}
    for sample in gts_classes[cat]:
        dts_temp[sample] = dts_lv1[sample]
        gts_temp[sample] = gts_lv1[sample]

    for key in gts_temp.keys():
        for i in range(len(gts_temp[key])):
            if dts_temp[key][i] == 1:
                pc_precision_total_1[cat] += 1
            if gts_temp[key][i] == 1:
                pc_total_1[cat] += 1
                if dts_temp[key][i] == 1:
                    pc_accurate_1[cat] +=1
pc_acc_1 = [pc_accurate_1[i]/pc_total_1[i] for i in range(397)]
pc_pre_1 = [round(pc_accurate_1[i]/pc_precision_total_1[i],2) for i in range(397)]
pc_f1_1 = [compute_f1(pc_acc_1[i],pc_pre_1[i]) for i in range(397)]


table_1 = [[classes[i],pc_acc_1[i],pc_pre_1[i],pc_f1_1[i]] for i in range(397)]
# def takeSecond(elem):
#     return elem[2]
# table.sort(key = takeSecond,reverse = False)
print(tabulate(table_1, headers=['class','level_1 recall', 'level_1 precision', 'level_1 f1']))

class                          level_1 recall    level_1 precision    level_1 f1
---------------------------  ----------------  -------------------  ------------
abbey                                    0.96                 0.91          0.93
airplane_cabin                           1                    1             1
airport_terminal                         1                    0.98          0.99
alley                                    0.94                 0.94          0.94
amphitheater                             1                    1             1
amusement_arcade                         1                    1             1
amusement_park                           0.98                 0.92          0.95
anechoic_chamber                         0.98                 0.98          0.98
apartment_building-outdoor               1                    1             1
apse-indoor                              0.98                 0.98          0.98
aquarium                                

# Level 2: Evaluation:

In [19]:
# mapping gts to level2
lv2_names = ['shopping and dining','workplace (office building, factory, lab, etc.)','home or hotel','transportation (vehicle interiors, stations, etc.)','sports and leisure','cultural (art, education, religion, millitary, law, politics, etc.)','water, ice, snow','mountains, hills, desert, sky','forest, field, jungle','man-made elements','transportation (roads, parking, bridges, boats, airports, etc.)','cultural or historical building/place (millitary, religious)','sports fields, parks, leisure spaces','industrial and construction','houses, cabins, gardens, and farms','commercial buildings, shops, markets, cities, and towns']
level2 = {}
with open('./SUN397_level2.csv', 'r') as file:
    reader = csv.reader(file)
    count = 0
    for row in reader:
        level2[count]=[int(row[a]) for a in range(len(row))]
        count +=1
# load the class label
classes = tuple(classes)

# load ground truth
gts_lv2 = []
for i in range(len(gts)):
    value = gts[i][0]
    gts_lv2.append(level2[value]) 
  
    
# mapping dts to level1:


temp = []
for label in level2.keys():
    if level2[label] not in temp:
        temp.append(level2[label])


lv2 = {}
for key in level2.keys():
    lv2[key] = temp.index(level2[key])
# lv1 mapping original class_id to level1_id  

lv2_classes = {}
for i in range(len(temp)):
    lv2_classes[i] = temp[i]

dts_lv2 = []
for i in range(len(dts)):
    temp =lv2_classes[lv2[dts[i][0]]]
    dts_lv2.append(temp)
    
top1,pc_total,pc_accurate,pc_precision_total = top1_accuracy_highLevel(dts_lv2,gts_lv2)
print('Top-1 accuracy of at level-2 is {}'.format(float(top1)))
for i in range(len(pc_total)):
    total = pc_total[i]
    acc = float(pc_accurate[i]/pc_total[i])
    name = lv2_names[i]
    print('{} samples in: {}. The accurate is {:.2f}'.format(total,name,acc))

Top-1 accuracy of at level-2 is 0.9000392156862745
2000 samples in: shopping and dining. The accurate is 0.91
2000 samples in: workplace (office building, factory, lab, etc.). The accurate is 0.90
1750 samples in: home or hotel. The accurate is 0.92
1050 samples in: transportation (vehicle interiors, stations, etc.). The accurate is 0.92
1100 samples in: sports and leisure. The accurate is 0.89
1800 samples in: cultural (art, education, religion, millitary, law, politics, etc.). The accurate is 0.89
1750 samples in: water, ice, snow. The accurate is 0.94
700 samples in: mountains, hills, desert, sky. The accurate is 0.92
1900 samples in: forest, field, jungle. The accurate is 0.92
1950 samples in: man-made elements. The accurate is 0.88
1350 samples in: transportation (roads, parking, bridges, boats, airports, etc.). The accurate is 0.88
1850 samples in: cultural or historical building/place (millitary, religious). The accurate is 0.88
2000 samples in: sports fields, parks, leisure spa

In [20]:
pc_total_2 = [0 for i in range(397)]
pc_accurate_2 = [0 for i in range(397)]
pc_precision_total_2 = [0 for i in range(397)]
for cat in gts_classes.keys():
    dts_temp = {}
    gts_temp = {}
    for sample in gts_classes[cat]:
        dts_temp[sample] = dts_lv2[sample]
        gts_temp[sample] = gts_lv2[sample]

    for key in gts_temp.keys():
        for i in range(len(gts_temp[key])):
            if dts_temp[key][i] == 1:
                pc_precision_total_2[cat] += 1
            if gts_temp[key][i] == 1:
                pc_total_2[cat] += 1
                if dts_temp[key][i] == 1:
                    pc_accurate_2[cat] +=1
pc_acc_2 = [pc_accurate_2[i]/pc_total_2[i] for i in range(397)]
pc_pre_2 = [round(pc_accurate_2[i]/pc_precision_total_2[i],2) for i in range(397)]
pc_f1_2 = [compute_f1(pc_acc_2[i],pc_pre_2[i]) for i in range(397)]


table_2 = [[classes[i],pc_acc_2[i],pc_pre_2[i],pc_f1_2[i]] for i in range(397)]
# def takeSecond(elem):
#     return elem[2]
# table.sort(key = takeSecond,reverse = False)
print(tabulate(table_2, headers=['class','level_2 recall', 'level_2 precision', 'level_2 f1']))

class                          level_2 recall    level_2 precision    level_2 f1
---------------------------  ----------------  -------------------  ------------
abbey                                0.82                     0.73          0.77
airplane_cabin                       0.98                     0.98          0.98
airport_terminal                     0.76                     0.69          0.72
alley                                0.92                     0.77          0.84
amphitheater                         0.98                     0.94          0.96
amusement_arcade                     0.88                     0.86          0.87
amusement_park                       0.82                     0.75          0.78
anechoic_chamber                     0.9                      0.9           0.9
apartment_building-outdoor           0.9                      0.85          0.87
apse-indoor                          0.98                     0.98          0.98
aquarium                     

# Generate Level 3

for each froup in level 2:
    - run error analysis to group classes with large overlap, give a name to each of the new group
    - run clustering based on w2v embedding
    - manually check

 - Filter out the classes achieving accuracy higher than 0.8

# Level3: Evaluation

In [21]:
level3 = {}
with open('./SUN397_level3.csv', 'r') as file:
    reader = csv.reader(file)
    count = -1
    for row in reader:
        level3[count]=[(row[a]) for a in range(len(row))]
        count +=1
level3_classes = level3[-1]
del level3[-1]
for key in level3.keys():
    level3[key] = [int(i) for i in level3[key]]
    
# level3_classes

In [22]:
# load ground truth
gts_lv3 = []
for i in range(len(gts)):
    value = gts[i][0]
    gts_lv3.append(level3[value]) 
  
    
# mapping dts to level1:


temp = []
for label in level3.keys():
    if level3[label] not in temp:
        temp.append(level3[label])


lv3 = {}
for key in level3.keys():
    lv3[key] = temp.index(level3[key])
# lv1 mapping original class_id to level1_id  

lv3_classes = {}
for i in range(len(temp)):
    lv3_classes[i] = temp[i]

dts_lv3 = []
for i in range(len(dts)):
    temp =lv3_classes[lv3[dts[i][0]]]
    dts_lv3.append(temp)

In [23]:
top1,pc_total,pc_accurate, pc_precision_total = top1_accuracy_highLevel(dts_lv3,gts_lv3)
print('Top-1 accuracy of at level-3 is {}'.format(float(top1)))
for i in range(len(pc_total)):
    total = pc_total[i]
    acc = float(pc_accurate[i]/pc_total[i])
    name = level3_classes[i]
    print('{} samples in: {}. The accurate is {:.2f}'.format(total,name,acc))

Top-1 accuracy of at level-3 is 0.871264367816092
1550 samples in: shop. The accurate is 0.88
900 samples in: restaurant. The accurate is 0.86
700 samples in: office. The accurate is 0.85
800 samples in: lab & factory. The accurate is 0.85
1750 samples in: home & hotel. The accurate is 0.92
350 samples in: airplane. The accurate is 0.86
800 samples in: vehicle & train & ship. The accurate is 0.91
750 samples in: sports. The accurate is 0.90
650 samples in: leisure. The accurate is 0.83
600 samples in: art & entertainment. The accurate is 0.84
350 samples in: education. The accurate is 0.70
500 samples in: religion. The accurate is 0.90
150 samples in: law. The accurate is 0.85
1350 samples in: indoor_others. The accurate is 0.83
400 samples in: ice & snow . The accurate is 0.95
1300 samples in: water. The accurate is 0.92
550 samples in: mountains & hills & canyon. The accurate is 0.90
100 samples in: dessert. The accurate is 0.92
50 samples in: sky. The accurate is 0.90
400 samples in

In [24]:
pc_total_3 = [0 for i in range(397)]
pc_accurate_3 = [0 for i in range(397)]
pc_precision_total_3 = [0 for i in range(397)]
for cat in gts_classes.keys():
    dts_temp = {}
    gts_temp = {}
    for sample in gts_classes[cat]:
        dts_temp[sample] = dts_lv3[sample]
        gts_temp[sample] = gts_lv3[sample]

    for key in gts_temp.keys():
        for i in range(len(gts_temp[key])):
            if dts_temp[key][i] == 1:
                pc_precision_total_3[cat] += 1
            if gts_temp[key][i] == 1:
                pc_total_3[cat] += 1
                if dts_temp[key][i] == 1:
                    pc_accurate_3[cat] +=1
pc_acc_3 = [pc_accurate_3[i]/pc_total_3[i] for i in range(397)]
pc_pre_3 = [round(pc_accurate_3[i]/pc_precision_total_3[i],2) for i in range(397)]
pc_f1_3 = [compute_f1(pc_acc_3[i],pc_pre_3[i]) for i in range(397)]


table_3 = [[classes[i],pc_acc_3[i],pc_pre_3[i],pc_f1_3[i]] for i in range(397)]
# def takeSecond(elem):
#     return elem[2]
# table.sort(key = takeSecond,reverse = False)
print(tabulate(table_3, headers=['class','level_3 recall', 'level_3 precision', 'level_3 f1']))

class                          level_3 recall    level_3 precision    level_3 f1
---------------------------  ----------------  -------------------  ------------
abbey                                0.82                     0.73          0.77
airplane_cabin                       0.92                     0.9           0.91
airport_terminal                     0.62                     0.5           0.55
alley                                0.92                     0.75          0.83
amphitheater                         0.98                     0.92          0.95
amusement_arcade                     0.88                     0.86          0.87
amusement_park                       0.82                     0.77          0.79
anechoic_chamber                     0.84                     0.82          0.83
apartment_building-outdoor           0.9                      0.85          0.87
apse-indoor                          0.98                     0.98          0.98
aquarium                    